#### @author: Александр Владимирович Толмачев | axtolm@gmail.com
<hr>

## 3. Предобработка данных на Python

## Часть 1. Работа с пропусками в данных

### На этом занятии мы планируем научиться:
- Получать для таблицы pandas DataFrame описательную статистику.
- Находить пропуски в данных и исправлять их.

### 1. Вводная информация
На практике может возникнуть ситуация, когда полученные из какого-либо источника данные содержат пропуски.<br>

Как выглядят пропуски в исходных данных (котировки акций Лукойл) в CSV формате:

```
<TICKER>;<PER>;<DATE>;<TIME>;<CLOSE>;<VOL>
LKOH;D;20171019;000000;2985.0000000;417461
LKOH;D;20171020;000000;2994.0000000;
LKOH;D;20171023;000000;;394534
LKOH;D;20171024;000000;3018.0000000;338596
...
```
Как отсутствующие элементы - для примера 2-я строка `<VOL>` и 3-я строка `<CLOSE>`

Пропуски в Python pandas DataFrame обозначаются `NaN` (как это выглядит - увидим ниже). Нужно уметь их находить и исправлять. 

### 2. Загрузка данных из файла в csv формате в таблицу pandas DataFrame

Для загрузки воспользуемся методом `pandas.read_csv`<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html<br>

In [1]:
# зададим имя файла с пропущенными данными (лежит в той же папке)
file_LKOH_NA = 'dataset_1_LKOH_NA.csv'

In [2]:
import pandas as pd    # импорт библиотеки

Загрузим в DataFrame файл данных в формате csv `file_LKOH_NA`, имеющий пропущенные данные. 
В нем котировки обыкновенных акций Лукойла.<br>
Т.к. файл лежит там же, где и `.ipynb`, то полный путь можно не указывать. 

In [3]:
df_LKOH_NA = pd.read_csv(file_LKOH_NA)

### 3. Обнаружение пропусков

In [4]:
df_LKOH_NA    # проверим, что получилось после загрузки

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
0,LKOH,D,20171019,0,2985.0,417461.0
1,LKOH,D,20171020,0,2994.0,424104.0
2,LKOH,D,20171023,0,2997.0,394534.0
3,LKOH,D,20171024,0,3018.0,338596.0
4,LKOH,D,20171025,0,3005.0,620960.0
...,...,...,...,...,...,...
1006,LKOH,D,20211013,0,7252.0,1257240.0
1007,LKOH,D,20211014,0,7215.0,804831.0
1008,LKOH,D,20211015,0,7306.5,950893.0
1009,LKOH,D,20211018,0,7319.0,711222.0


По началу и концу таблицы не видно ничего необычного, но там более 1000 строк.<br>

Проверим таблицу с помощью метода `pandas.DataFrame.describe`, который считает описательную статистику.<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

In [5]:
df_LKOH_NA.describe()

,<DATE>,<TIME>,<CLOSE>,<VOL>
count,1.011000e+03,1011.0,1005.000000,1.006000e+03
mean,2.019366e+07,0.0,5180.651244,1.054897e+06
std,1.184013e+04,0.0,920.365528,7.358315e+05
min,2.017102e+07,0.0,2985.000000,5.365800e+04
25%,2.018102e+07,0.0,4519.500000,6.158762e+05
50%,2.019102e+07,0.0,5206.000000,8.393800e+05
75%,2.020102e+07,0.0,5868.500000,1.282734e+06
max,2.021102e+07,0.0,7346.500000,7.510762e+06


Что видим? Есть пропуски в данных таблицы (разные значения count в столбцах) - см. колонки `<CLOSE>` и `<VOL>`.<br>
Дата воспринимается как число и обрабатывается как число. Данные по объемам представлены в экспоненциальной форме записи.

Преобразуем дату из числа во временной формат с помощью метода `pandas.to_datetime`(формат `%Y%m%d` укажем явно )<br>
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [6]:
df_LKOH_NA['<DATE>'] = pd.to_datetime(df_LKOH_NA['<DATE>'],format = '%Y%m%d')

Объемы будем преобразовывать из экспоненциальной формы в обычную числовую с помощью метода `pandas.round` сразу при выводе на экран с точностью до 2 знаков после запятой<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html

In [7]:
df_LKOH_NA.describe().round(decimals=2)

,<TIME>,<CLOSE>,<VOL>
count,1011.0,1005.00,1006.00
mean,0.0,5180.65,1054896.84
std,0.0,920.37,735831.54
min,0.0,2985.00,53658.00
25%,0.0,4519.50,615876.25
50%,0.0,5206.00,839380.00
75%,0.0,5868.50,1282734.50
max,0.0,7346.50,7510762.00


Чтобы удостовериться в наличии пропусков наверняка, воспользуемся методом `pandas.isna`, который возвращает таблицу pandas DataFrame, аналогичную по размеру исходной, но содержащую True (если есть пропуск) и False (если пропуска нет).<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html#pandas.isna

In [8]:
df_LKOH_NA.isna()

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
1006,False,False,False,False,False,False
1007,False,False,False,False,False,False
1008,False,False,False,False,False,False
1009,False,False,False,False,False,False


Чтобы найти конкретные строки, где есть пропуски (NaN), воспользуемся фильтрацией с использованием методов `isna` и `.isin`<br> 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html<br> 
`isna` вернет True, если пропуск есть, False - если нет. `.isin` вернет True, если элемент входит в список в скобках.

In [9]:
df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([True])]    # пропуски в колонке <CLOSE>

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,NaN,330965.0
63,LKOH,D,2018-01-22,0,NaN,610914.0
228,LKOH,D,2018-09-13,0,NaN,586466.0
293,LKOH,D,2018-12-14,0,NaN,733967.0
789,LKOH,D,2020-12-07,0,NaN,1184748.0
889,LKOH,D,2021-04-30,0,NaN,NaN


In [10]:
df_LKOH_NA[df_LKOH_NA.isna()['<VOL>'].isin([True])]    # пропуски в колонке <VOL>

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
122,LKOH,D,2018-04-17,0,3940.0,NaN
169,LKOH,D,2018-06-22,0,4125.0,NaN
313,LKOH,D,2019-01-16,0,5129.0,NaN
827,LKOH,D,2021-02-02,0,5516.0,NaN
889,LKOH,D,2021-04-30,0,NaN,NaN


Полученные результаты можно сохранить в таблицу pandas DataFrame. Из нее можно взять индексы строк, где есть пропуски, для их устранения в исходной таблице pandas DataFrame. 

### 4. Исправление пропусков

**Что будем делать с пропусками в данных?**<br><br>
**Вариант 1.** Удалить строки где есть пропуски - вариант не очень хорош, т.к. в случае временного ряда промежутки времени станут не одинаковыми и ряд перестанет быть равномерным.<br><br>
**Вариант 2.** Заменить пропуски на какие-либо значения. В случае временного ряда это могут быть, например, средние значения показателя по предыдущему и последующему непропущенным элементам, линейная или полиномиальная интерполяция. Если это выборка на один момент времени, усреднять можно по всей выборке и делать замену на такое среднее. Также можно выполнить замену пропусков предыдущими или последующими непропущенными значениями.

**Как будем исправлять пропуски в данных?**<br><br>
**Способ 1.** Написать немного своего кода на Python, который будет отбирать пропуски и выполнять над ними необходимые действия.<br><br>
**Способ 2.** Использовать методы `dropna`, `fillna` и другие из библиотеки `pandas`.<br> 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html<br>

**Вариант 1. Удаление пропусков из таблицы pandas DataFrame**

**Способ 1. Исключение пропусков путем фильтрации с помощью своего кода.**

Для этого воспользуемся логической конструкцией "И" = "&", объединив два условия.   
В качестве аргумента `isin()` возьмем `False`, т.к. нам нужно выбрать строки, в которых нет пропусков.

In [11]:
df_LKOH_1_1 = df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([False]) & df_LKOH_NA.isna()['<VOL>'].isin([False])]

In [12]:
df_LKOH_1_1.describe().round(decimals=2)

,<TIME>,<CLOSE>,<VOL>
count,1001.0,1001.00,1001.00
mean,0.0,5182.66,1056722.44
std,0.0,920.70,736947.40
min,0.0,2985.00,53658.00
25%,0.0,4520.00,618624.00
50%,0.0,5209.00,840943.00
75%,0.0,5873.00,1283942.00
max,0.0,7346.50,7510762.00


Для проверки аналогичным образом отфильтруем таблицу с пропусками. Вместо "И" будет "ИЛИ" ("|") и вместо `False` будет `True`.

In [13]:
df_LKOH_1_1_NA_only = df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([True]) | df_LKOH_NA.isna()['<VOL>'].isin([True])]
df_LKOH_1_1_NA_only    # выведем таблицу из строк с пропусками данных на экран

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,NaN,330965.0
63,LKOH,D,2018-01-22,0,NaN,610914.0
122,LKOH,D,2018-04-17,0,3940.0,NaN
169,LKOH,D,2018-06-22,0,4125.0,NaN
228,LKOH,D,2018-09-13,0,NaN,586466.0
293,LKOH,D,2018-12-14,0,NaN,733967.0
313,LKOH,D,2019-01-16,0,5129.0,NaN
789,LKOH,D,2020-12-07,0,NaN,1184748.0
827,LKOH,D,2021-02-02,0,5516.0,NaN
889,LKOH,D,2021-04-30,0,NaN,NaN


In [14]:
df_LKOH_1_1_NA_only.describe().round(decimals=2)    # покажем описательную статистику

,<TIME>,<CLOSE>,<VOL>
count,10.0,4.00,5.0
mean,0.0,4677.50,689412.0
std,0.0,765.09,313283.8
min,0.0,3940.00,330965.0
25%,0.0,4078.75,586466.0
50%,0.0,4627.00,610914.0
75%,0.0,5225.75,733967.0
max,0.0,5516.00,1184748.0


У нас было 10 строк с пропусками в одной из колонок `'<CLOSE>'`или `'<VOL>'`или в обеих сразу.<br>
В итоговой таблице их не стало и строк стало на 10 меньше.

**Способ 1. Удаление пропусков с помощью своего кода.**

In [15]:
# Сделаем копию, чтобы удалять в ней
df_LKOH_1_2 = df_LKOH_NA.copy()

In [16]:
# получим в виде списка индексы строк, в которых есть пропуски и которые надо удалить
# конструкция .index.tolist() выделит индексную колонку в отфильтрованной таблице DataFrame и преобразует ее в список
del_list = df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([True]) | df_LKOH_NA.isna()['<VOL>'].isin([True])].index.tolist()
del_list    # выведем список на экран

[20, 63, 122, 169, 228, 293, 313, 789, 827, 889]

In [17]:
df_LKOH_1_2 = df_LKOH_1_2.drop(del_list)   # удалим строки с индексами из списка и сохраним результат в ту же таблицу
df_LKOH_1_2.describe().round(decimals=2)    # покажем описательную статистку

,<TIME>,<CLOSE>,<VOL>
count,1001.0,1001.00,1001.00
mean,0.0,5182.66,1056722.44
std,0.0,920.70,736947.40
min,0.0,2985.00,53658.00
25%,0.0,4520.00,618624.00
50%,0.0,5209.00,840943.00
75%,0.0,5873.00,1283942.00
max,0.0,7346.50,7510762.00


Как и в прошлом способе строк стало на 10 меньше.

**Способ 2. Удаление пропусков с помощью `pandas.dropna`.**

In [18]:
# Сделаем копию, чтобы удалять в ней
df_LKOH_1_3 = df_LKOH_NA.copy()

In [19]:
df_LKOH_1_3 = df_LKOH_1_3.dropna(axis = 'index')   # удалим строки с пропусками, если axis = 'columns' - столбцы
df_LKOH_1_3.describe().round(decimals=2)    # покажем описательную статистку

,<TIME>,<CLOSE>,<VOL>
count,1001.0,1001.00,1001.00
mean,0.0,5182.66,1056722.44
std,0.0,920.70,736947.40
min,0.0,2985.00,53658.00
25%,0.0,4520.00,618624.00
50%,0.0,5209.00,840943.00
75%,0.0,5873.00,1283942.00
max,0.0,7346.50,7510762.00


Аналогичный вариант был получен нами Способом 1.

**Вариант 2. Замена пропусков в таблице pandas DataFrame**

**Способ 1. Замена пропусков на средние значения по предыдущему и последующему непропущенным элементам с помощью своего кода**

In [20]:
# Сделаем копию, чтобы менять в ней
df_LKOH_1_4 = df_LKOH_NA.copy()

Получим в виде списка индексы строк, в которых есть пропуски и которые надо заменить на средние.<br>
Конструкция `.index.tolist` выделит индексную колонку в отфильтрованной таблице DataFrame и преобразует ее в список.

Начнем с колонки `'<CLOSE>'` и обработаем колонки по отдельности.

In [21]:
change_list_close = df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([True])].index.tolist()
change_list_close    # выведем список на экран

[20, 63, 228, 293, 789, 889]

In [22]:
df_LKOH_NA[df_LKOH_NA.index.isin(change_list_close)]    # Выведем строки с пропусками на экран

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,NaN,330965.0
63,LKOH,D,2018-01-22,0,NaN,610914.0
228,LKOH,D,2018-09-13,0,NaN,586466.0
293,LKOH,D,2018-12-14,0,NaN,733967.0
789,LKOH,D,2020-12-07,0,NaN,1184748.0
889,LKOH,D,2021-04-30,0,NaN,NaN


Сделаем цикл по всем элементам списка замены.<br>
В исходной таблице индексы идут по порядку с 0, но мы для общности будем вычислять порядковый номер `i` с помощью метода `pandas.Index.get_loc`, который по значению индекса возвращает его порядковый номер.<br>
https://pandas.pydata.org/docs/reference/api/pandas.Index.get_loc.html<br>
Индекс предыдущего будет `i-1`, а последующего `i+1`.<br>
Замену пропуска в строке `i` надо делать по формуле: $close_i = \frac{1}{2}(close_{i-1} + close_{i+1})$.<br>
Такая замена является линейной интерполяцией пропущенных данных.

> В общем случе надо еще проверять, чтобы заменяемый таким образом элемент не был первым и последним в выборке, а также, чтобы не было двух и более пропусков подряд. Для них нужна другая формула. Какая? - предложение подумать слушателям.<br>

In [23]:
for i in change_list_close:
    i_iloc = df_LKOH_1_4.index.get_loc(i)    # вычисляем порядковый номер по значению индекса
    new_value = (df_LKOH_1_4.iloc[i_iloc-1]['<CLOSE>'] + df_LKOH_1_4.iloc[i_iloc+1]['<CLOSE>'])/2    # новое значение для NaN
    df_LKOH_1_4.loc[i,'<CLOSE>'] = new_value    # присваиваем новое значение для элемента строки, где был пропуск Nan    

In [24]:
df_LKOH_1_4[df_LKOH_1_4.index.isin(change_list_close)]    # Выведем строки, где были пропуски, на экран

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,3292.50,330965.0
63,LKOH,D,2018-01-22,0,3833.50,610914.0
228,LKOH,D,2018-09-13,0,4613.25,586466.0
293,LKOH,D,2018-12-14,0,5115.75,733967.0
789,LKOH,D,2020-12-07,0,5058.00,1184748.0
889,LKOH,D,2021-04-30,0,5958.50,NaN


Замена пропусков состоялась.

Аналогичным образом поступим с колонкой `'<VOL>'`.

In [25]:
change_list_vol = df_LKOH_NA[df_LKOH_NA.isna()['<VOL>'].isin([True])].index.tolist()
change_list_vol    # выведем список на экран

[122, 169, 313, 827, 889]

In [26]:
for i in change_list_vol:
    i_iloc = df_LKOH_1_4.index.get_loc(i)    # вычисляем порядковый номер по значению индекса
    new_value = (df_LKOH_1_4.iloc[i_iloc-1]['<VOL>'] + df_LKOH_1_4.iloc[i_iloc+1]['<VOL>'])/2    # новое значение для NaN
    df_LKOH_1_4.loc[i,'<VOL>'] = new_value    # присваиваем новое значение для элемента строки, где был пропуск Nan    

Почему обрабатывали колонки по отдельности? Если сделать объединенный список, то будут строки, в которых в `'<CLOSE>'` пропуск (его надо исправлять), а в `'<VOL>'` его нет (значение надо оставить как есть). Проверку на такие ситуации нужно будет делать в цикле. Чтобы ее не делать, обработали колонки по отдельности и исключили такие ситуации.

In [27]:
df_LKOH_1_4.describe().round(decimals=2)    # выведем описательную статистику

,<TIME>,<CLOSE>,<VOL>
count,1011.0,1011.00,1011.00
mean,0.0,5177.47,1054942.44
std,0.0,921.03,734830.03
min,0.0,2985.00,53658.00
25%,0.0,4516.00,615884.50
50%,0.0,5204.50,839558.00
75%,0.0,5866.75,1282568.00
max,0.0,7346.50,7510762.00


Количество элементов теперь как в исходном DataFrame - пропусков данных нет. Пропуски в случае временного ряда мы заполнили средними значениями по ближайшим соседям.

**Способ 2. Замена с помощью `pandas.fillna` на предыдущие непропущенные значения**

In [28]:
# Сделаем копию, чтобы менять в ней
df_LKOH_1_5 = df_LKOH_NA.copy()

In [29]:
# Получим в виде списка индексы строк, в которых есть пропуски и которые надо заменить на средние
change_list_close = df_LKOH_NA[df_LKOH_NA.isna()['<CLOSE>'].isin([True])].index.tolist()
change_list_close    # выведем список на экран

[20, 63, 228, 293, 789, 889]

In [30]:
df_LKOH_NA[df_LKOH_NA.index.isin(change_list_close)]    # Выведем строки с пропусками на экран

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,NaN,330965.0
63,LKOH,D,2018-01-22,0,NaN,610914.0
228,LKOH,D,2018-09-13,0,NaN,586466.0
293,LKOH,D,2018-12-14,0,NaN,733967.0
789,LKOH,D,2020-12-07,0,NaN,1184748.0
889,LKOH,D,2021-04-30,0,NaN,NaN


In [31]:
# Сделаем замену пропусков предыдущими непропущенными значениями с помощью метода fillna 
# Параметр method = 'ffill' - замена вперед
df_LKOH_1_5 = df_LKOH_1_5.fillna(method = 'ffill')
df_LKOH_1_5[df_LKOH_1_5.index.isin(change_list_close)]    # выведем на экран строки, где были пропуски

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
20,LKOH,D,2017-11-17,0,3304.0,330965.0
63,LKOH,D,2018-01-22,0,3825.5,610914.0
228,LKOH,D,2018-09-13,0,4622.5,586466.0
293,LKOH,D,2018-12-14,0,5131.5,733967.0
789,LKOH,D,2020-12-07,0,5122.0,1184748.0
889,LKOH,D,2021-04-30,0,5926.0,1022280.0


In [32]:
# Для проверки составим список предыдущих значений с использованием генератора списков
change_list_close_backfill = [(i-1)for i in change_list_close]
df_LKOH_1_5[df_LKOH_1_5.index.isin(change_list_close_backfill)]    # выведем на экран предыдущие значения

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>,<VOL>
19,LKOH,D,2017-11-16,0,3304.0,580867.0
62,LKOH,D,2018-01-19,0,3825.5,662103.0
227,LKOH,D,2018-09-12,0,4622.5,586148.0
292,LKOH,D,2018-12-13,0,5131.5,901654.0
788,LKOH,D,2020-12-04,0,5122.0,1507715.0
888,LKOH,D,2021-04-29,0,5926.0,1022280.0


На место пропусков встали предыдущие непропущенные значения. Если встроенные методы замены устраивают, то метод `fillna` хорош.

Для интерполяции есть свой метод `pandas.DataFrame.interpolate`. С ним можете познакомиться самостоятельно.

### Подведем итоги. На этом занятии мы научились:
- Получать для таблицы pandas DataFrame описательную статистику.
- Находить пропуски в данных и исправлять их.